# Analysis

In [31]:
import json

with open('diffs.json', 'r') as f:
    diffs = json.load(f)

with open('codes.json', 'r') as cf:
    codes = json.load(cf)

## Dataset

In [32]:
def repo_name(diff: dict[str]) -> str:
    _, _, _, owner, repo, *_ = diff['url'].split('/')
    return f'{owner}/{repo}'

TOTAL = 538
TOTAL_REPOS = 434

retrieved = len(diffs)
retrieved_repos = len(set(repo_name(d) for d in diffs))
print('retrieved commits:', f'{retrieved}/{TOTAL}', f'({retrieved/TOTAL*100:.2f}%)')
print('retrieved repos:', f'{retrieved_repos}/{TOTAL_REPOS}', f'({retrieved_repos/TOTAL_REPOS*100:.2f}%)')

with_saving = len([d for d in diffs if 'saving' in d['existing_codes']])
print('\noriginally coded with `saving`:', f'{with_saving}/{retrieved}', f'({with_saving/retrieved*100:.2f}%)')

n_processed = sum(len(d['codes']) != 0 for d in diffs)
n_processed_with_saving = sum(len(d['codes']) != 0 and 'saving' in d['existing_codes'] for d in diffs)
print('\nprocessed commits:', f'{n_processed}/{retrieved}', f'({n_processed/retrieved*100:.2f}%)')
print('processed commits originally coded with `saving`:', f'{n_processed_with_saving}/{with_saving}', f'({n_processed_with_saving/with_saving*100:.2f}%)')

no_cost_change = sum('no_cost_change' in d['codes'] for d in diffs)
print('\ncommits coded with `no_cost_change`:', f'{no_cost_change}/{n_processed}', f'({no_cost_change/retrieved*100:.2f}%)')

avg_files = sum(len(d['files']) for d in diffs) / len(diffs)
print('\nfiles affected (avg):', f'{avg_files:.2f}')

retrieved commits: 499/538 (92.75%)
retrieved repos: 403/434 (92.86%)

originally coded with `saving`: 342/499 (68.54%)

processed commits: 499/499 (100.00%)
processed commits originally coded with `saving`: 342/342 (100.00%)

commits coded with `no_cost_change`: 179/499 (35.87%)

files affected (avg): 4.42


## Codes

In [33]:
import polars as pl

codes_df = pl.DataFrame(codes) \
            .with_columns(count=sum(pl.col('name').is_in(d['codes']) for d in diffs)) \
            .with_columns(percentage=pl.col('count') / n_processed * 100) \
            .sort(by=pl.col('count'), descending=True)

print(codes_df)

codes_df.drop("percentage").write_csv("codes.csv")

shape: (128, 4)
┌─────────────────────────────┬───────────────────────────────────┬───────┬────────────┐
│ name                        ┆ description                       ┆ count ┆ percentage │
│ ---                         ┆ ---                               ┆ ---   ┆ ---        │
│ str                         ┆ str                               ┆ i32   ┆ f64        │
╞═════════════════════════════╪═══════════════════════════════════╪═══════╪════════════╡
│ aws                         ┆ Specific to Amazon Web Services   ┆ 363   ┆ 72.745491  │
│ no_cost_change              ┆ The commit contents do not refle… ┆ 179   ┆ 35.871743  │
│ cheaper_instance            ┆ Use a cheaper compute instance    ┆ 74    ┆ 14.829659  │
│ gcp                         ┆ Specific to Google Cloud Platfor… ┆ 64    ┆ 12.825651  │
│ azure                       ┆ Specific to Azure                 ┆ 52    ┆ 10.420842  │
│ …                           ┆ …                                 ┆ …     ┆ …          │
│ red

### Cloud-specific Codes

In [34]:
clouds = {'digitalocean', 'aws', 'gcp', 'azure', 'alicloud', 'scaleway', 'hcloud', 'ibm', 'oracle', 'ovh'}

cloud_codes = codes_df.filter(pl.col('name').is_in(clouds)).drop('description')

print(cloud_codes)

# for name, count, pct in cloud_codes.iter_rows():
#     print(f'{name} & {count} & {pct:.1f}\\%', '\\\\\n\\midrule')

non_cloud_codes = codes_df.filter(pl.col('name').is_in(clouds).not_()).drop('description')

print(non_cloud_codes)

print(cloud_codes.select('count').sum())
for d in diffs:
    n_clouds = len(set(d['codes']) & clouds)
    if n_clouds != 1:
        print(d['url'], d['codes'])

shape: (10, 3)
┌──────────────┬───────┬────────────┐
│ name         ┆ count ┆ percentage │
│ ---          ┆ ---   ┆ ---        │
│ str          ┆ i32   ┆ f64        │
╞══════════════╪═══════╪════════════╡
│ aws          ┆ 363   ┆ 72.745491  │
│ gcp          ┆ 64    ┆ 12.825651  │
│ azure        ┆ 52    ┆ 10.420842  │
│ digitalocean ┆ 12    ┆ 2.40481    │
│ scaleway     ┆ 3     ┆ 0.601202   │
│ hcloud       ┆ 2     ┆ 0.400802   │
│ ibm          ┆ 2     ┆ 0.400802   │
│ oracle       ┆ 2     ┆ 0.400802   │
│ alicloud     ┆ 1     ┆ 0.200401   │
│ ovh          ┆ 1     ┆ 0.200401   │
└──────────────┴───────┴────────────┘
shape: (118, 3)
┌─────────────────────────────┬───────┬────────────┐
│ name                        ┆ count ┆ percentage │
│ ---                         ┆ ---   ┆ ---        │
│ str                         ┆ i32   ┆ f64        │
╞═════════════════════════════╪═══════╪════════════╡
│ no_cost_change              ┆ 179   ┆ 35.871743  │
│ cheaper_instance            ┆ 74    ┆ 14.

### Uniqueness

Determine how many unique owners the code occurs for, to see if there is actually a pattern across developers/organizations.

In [35]:
def repo_owner(diff: dict[str, str]) -> str:
    _, _, _, owner, *_ = diff['url'].split('/')
    return owner

n_owners = len(set(repo_owner(d) for d in diffs))
print('number of unique owners:', n_owners)

owners = [
    {
        'name': c['name'],
        'owners': list(set(repo_owner(d) for d in diffs if c['name'] in d['codes']))
    }
    for c in codes
]
owners_df = pl.DataFrame(owners) \
                .with_columns(n_owners=pl.col('owners').list.len()) \
                .sort(by=pl.col('n_owners'), descending=True)

filtered = owners_df.filter(pl.col('name').is_in(clouds).not_()).filter(pl.col('n_owners') >= 2)

print(filtered)

#filtered.drop('owners').write_csv('unique_owners.csv')

number of unique owners: 352
shape: (44, 3)
┌────────────────────────────┬───────────────────────────────────┬──────────┐
│ name                       ┆ owners                            ┆ n_owners │
│ ---                        ┆ ---                               ┆ ---      │
│ str                        ┆ list[str]                         ┆ u32      │
╞════════════════════════════╪═══════════════════════════════════╪══════════╡
│ no_cost_change             ┆ ["wushin", "CPWu", … "jjffggpp"]  ┆ 124      │
│ cheaper_instance           ┆ ["ayltai", "lowflying", … "deadl… ┆ 62       │
│ new_gen                    ┆ ["lowflying", "kmishra9", … "dea… ┆ 25       │
│ dynamo_on_demand           ┆ ["telia-oss", "olliefr", … "kper… ┆ 19       │
│ remove_nat_gateway         ┆ ["stealthHat", "HarsheshShah08",… ┆ 18       │
│ …                          ┆ …                                 ┆ …        │
│ reduce_node_count          ┆ ["giantswarm", "armand1m"]        ┆ 2        │
│ remove_status_chec

### Undefined codes

List codes which are attached to commits but not defined in `codes.json`.

In [36]:
listed_codes = set(c['name'] for c in codes)
for d in diffs:
    for c in d['codes']:
        if c not in listed_codes:
            print(f'{c} is not listed')